# 문자 단위 RNN (char RNN)
모든 시점의 입력에 대해서 출력하는 many-to-many RNN 구현

## 문자 단위 RNN(Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 데이터 전처리

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

문자들에 고유한 정수를 부여

In [3]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
print(char_set)
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic) # 공백도 여기서는 하나의 원소
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

['.', 'b', 'l', 'c', 'i', 't', 'g', 'k', ' ', 'f', 'e', 'a', 'p', 'y', 'd', 'h', ',', "'", 'o', 'n', 'm', 'u', 'w', 's', 'r']
{'.': 0, 'b': 1, 'l': 2, 'c': 3, 'i': 4, 't': 5, 'g': 6, 'k': 7, ' ': 8, 'f': 9, 'e': 10, 'a': 11, 'p': 12, 'y': 13, 'd': 14, 'h': 15, ',': 16, "'": 17, 'o': 18, 'n': 19, 'm': 20, 'u': 21, 'w': 22, 's': 23, 'r': 24}
문자 집합의 크기 : 25


문자 집합의 크기 = 입력으로 사용할 원핫 벡터의 크기

문장을 임의의 sequence_length로 잘라서 데이터를 만듬

In [6]:
sequence_length = 10  # 임의 숫자 지정

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

print(x_data[0])
print(y_data[0])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

원핫 인코딩을 수행

In [7]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩


데이터를 텐서로 변환

In [8]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))
print(X[0])
print(Y[0])


훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.

### 하이퍼 파라미터

In [15]:
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

### 모델 구현

In [16]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.


In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [18]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [19]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [20]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
                                                                                                                                                                                   
l.dsi..ttggggggggggggggggg...ggggggggggggggggggggggggggg..gggg.gg.ggg.gg....gggggg.ggg.gggggggggggggggg.gggggggggg.ggggggggggggggg.ggg.gggggg.gggggggggggggggg.ggg...gggggggggggggg
tocttacttaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
 tkt okt owkokwkwkpkkppkkkaokkkkwkookkpokppkkkpkkoakokkookkpookookkkppkakpokkwkpaowkkpkkookopkkkkookkookkkokkapkkpkkowkookkwakkkookkkpkkakkookkkokkpwowkookkppkkkkapkkkkkaokkkkwpko
 det tee ttetetetettetteeettteeetatttettetteeettetteteetteetttetteeettetetteatetttteeteettetteeeette